# Load Packages

In [1]:
using CSV
using DataFrames
using Gurobi
using Random
using Distances
using JuMP
using Gurobi

const GRB_ENV = Gurobi.Env(output_flag=1);

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2553220
Academic license 2553220 - for non-commercial use only - registered to ga___@mit.edu


# Load the data

In [ ]:
model = Model(Gurobi.Optimizer)

# Load data
full_pairs = CSV.read("./data/kidney_data_with_states.csv", DataFrame)
centers = CSV.read("./data/Transplant_Center_Dataset.csv", DataFrame)

# # Take random samples
pair_size = 100
# center_size = 6
Random.seed!(42)
pairs = full_pairs[1:100, :]
altruistic_donors = pairs[pairs.Altruist .== 1, :]

n = nrow(pairs)
m = nrow(centers)

println("Number of pairs: ", n)
println("Number of centers: ", m)

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2553220
Academic license 2553220 - for non-commercial use only - registered to ga___@mit.edu
Number of pairs: 100
Number of centers: 256


# Base Model

In [ ]:
# Create Compatibility Matrix Based on Exact Blood Type Matches
c = zeros(Int, n, n)  # Initialize n x n matrix with zeros

# Create compatibility matrix (1 if compatible, 0 otherwise)
for i in 1:n
    donor = pairs.Donor[i]
    for j in 1:n
        patient = pairs.Patient[j]
        # Exact blood type match
        c[i, j] = donor == patient ? 1 : 0
    end
end

# Create JuMP Model with Gurobi Optimizer
model = Model(Gurobi.Optimizer)
set_optimizer_attribute(model, "OutputFlag", 1) 

# Define Decision Variables: y[i,j] = 1 if donor i donates to patient j
@variable(model, y[1:n, 1:n], Bin)

# Define Objective: Maximize the Total Number of Transplants
@objective(model, Max, sum(y[i, j] for i in 1:n for j in 1:n))

# Add Compatibility Constraints: y[i,j] <= c[i,j]
@constraint(model, [i=1:n, j=1:n], y[i, j] <= c[i, j])

# Each Donor Can Donate to At Most One Patient: Σ y[i,j] <= 1, ∀ i
@constraint(model, [i=1:n], sum(y[i, j] for j in 1:n) <= 1)

# Each Patient Can Receive at Most One Kidney: Σ y[i,j] <= 1, ∀ j
@constraint(model, [j=1:n], sum(y[i, j] for i in 1:n) <= 1)

# Reciprocal Donation Constraints: Σ y[i,j] == Σ y[j,i], ∀ i
@constraint(model, [i=1:n], sum(y[i, j] for j in 1:n) == sum(y[j, i] for j in 1:n))

# Optimize the Model
optimize!(model)

# Check Optimization Status and Extract Results
status = termination_status(model)
if status == MOI.OPTIMAL
    println("\nOptimal solution found:\n")
    transplants = DataFrame(Donor = Int[], Patient = Int[])
    for i in 1:n
        for j in 1:n
            if value(y[i, j]) > 0.5
                println("Donor $(i) donates to Patient $(j)")
                push!(transplants, (Donor = i, Patient = j))
            end
        end
    end
    println("\nTotal optimal transplants: ", nrow(transplants))
    # display(transplants)
else
    println("\nNo optimal solution found. Status: ", status)
end

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2553220
Academic license 2553220 - for non-commercial use only - registered to ga___@mit.edu
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 9 7940HS w/ Radeon 780M Graphics, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Academic license 2553220 - for non-commercial use only - registered to ga___@mit.edu
Optimize a model with 10300 rows, 10000 columns and 49800 nonzeros
Model fingerprint: 0x06bb70a9
Variable types: 0 continuous, 10000 integer (10000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 16.0000000
Presolve removed 10026 rows and 7726 columns
Presolve time: 0.03s
Presolved: 274 rows, 2274 columns, 8409 nonzeros
Varia

# Base Model with Altrustic Donors

In [9]:
# Create Compatibility Matrix Based on Exact Blood Type Matches
c = zeros(Int, n, n)  # Initialize n x n matrix with zeros

# Create compatibility matrix (1 if compatible, 0 otherwise), 
for i in 1:n
    donor = pairs.Donor[i]
    altruistic = pairs.Altruist[i]  # Check if the donor is altruistic
    for j in 1:n
        patient = pairs.Patient[j]
        # Compatibility logic: altruistic donors can donate to all patients
        c[i, j] = altruistic == 1 ? 1 : (donor == patient ? 1 : 0)
    end
end

# Create JuMP Model with Gurobi Optimizer
model = Model(Gurobi.Optimizer)
set_optimizer_attribute(model, "OutputFlag", 1) 

# Define Decision Variables: y[i,j] = 1 if donor i donates to patient j
@variable(model, y[1:n, 1:n], Bin)

# Define Objective: Maximize the Total Number of Transplants
@objective(model, Max, sum(y[i, j] for i in 1:n for j in 1:n))

# Add Compatibility Constraints: y[i,j] <= c[i,j]
@constraint(model, [i=1:n, j=1:n], y[i, j] <= c[i, j])

# Each Donor Can Donate to At Most One Patient: Σ y[i,j] <= 1, ∀ i
@constraint(model, [i=1:n], sum(y[i, j] for j in 1:n) <= 1)

# Each Patient Can Receive at Most One Kidney: Σ y[i,j] <= 1, ∀ j
@constraint(model, [j=1:n], sum(y[i, j] for i in 1:n) <= 1)

# Reciprocal Donation Constraints: Σ y[i,j] == Σ y[j,i], ∀ i
@constraint(model, [i=1:n], sum(y[i, j] for j in 1:n) == sum(y[j, i] for j in 1:n))

# Optimize the Model
optimize!(model)

# Check Optimization Status and Extract Results
status = termination_status(model)
if status == MOI.OPTIMAL
    println("\nOptimal solution found:\n")
    transplants = DataFrame(Donor = Int[], Patient = Int[])
    for i in 1:n
        for j in 1:n
            if value(y[i, j]) > 0.5
                println("Donor $(i) donates to Patient $(j)")
                push!(transplants, (Donor = i, Patient = j))
            end
        end
    end
    println("\nTotal optimal transplants: ", nrow(transplants))
    # display(transplants)
else
    println("\nNo optimal solution found. Status: ", status)
end

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2553220
Academic license 2553220 - for non-commercial use only - registered to ga___@mit.edu
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 9 7940HS w/ Radeon 780M Graphics, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Academic license 2553220 - for non-commercial use only - registered to ga___@mit.edu
Optimize a model with 10300 rows, 10000 columns and 49800 nonzeros
Model fingerprint: 0x06bb70a9
Variable types: 0 continuous, 10000 integer (10000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 16.0000000
Presolve removed 10026 rows and 7726 columns
Presolve time: 0.03s
Presolved: 274 rows, 2274 columns, 8409 nonzeros
Varia

# Introducing Matching Score

In [11]:
# Create a compatibility dictionary with matching scores
comp = Dict()
compatibility_scores = Dict(
    ("O", "O") => 10, ("O", "A") => 8, ("O", "B") => 8, ("O", "AB") => 5,
    ("A", "A") => 10, ("A", "AB") => 7,
    ("B", "B") => 10, ("B", "AB") => 7,
    ("AB", "AB") => 10
)

# Populate the compatibility and matching scores
for i in 1:n, j in 1:n
    donor = pairs[i, :Donor]
    patient = pairs[j, :Patient]
    altruistic = pairs[i, :Altruist]
    
    # If donor is altruistic, assign a higher default score for all matches
    if altruistic == 1
        comp[(i, j)] = 15  # Higher score for altruistic donors
    else
        # Otherwise, use blood type matching scores or default to 0
        comp[(i, j)] = get(compatibility_scores, (donor, patient), 0)
    end
end

# Create JuMP model with Gurobi optimizer
model = Model(Gurobi.Optimizer)

# Define decision variables: y[i,j] = 1 if donor i donates to patient j
@variable(model, y[1:n, 1:n], Bin)

# Define objective: Maximize the total matching score
@objective(model, Max, sum(comp[(i, j)] * y[i, j] for i in 1:n, j in 1:n))

# Add compatibility constraints: y[i,j] <= 1 if compatible, else 0
@constraint(model, [i=1:n, j=1:n], y[i, j] <= (comp[(i, j)] > 0 ? 1 : 0))

# Each donor can donate to at most one patient
@constraint(model, [i=1:n], sum(y[i, j] for j in 1:n) <= 1)

# Each patient can receive from at most one donor
@constraint(model, [j=1:n], sum(y[i, j] for i in 1:n) <= 1)

# Optimize the model
optimize!(model)

# Check optimization status and print results
status = termination_status(model)
if status == MOI.OPTIMAL
    println("\nOptimal solution found with maximum matching score:\n")
    total_score = 0
    matches = DataFrame(Donor = Int[], Patient = Int[], Score = Int[])
    for i in 1:n
        for j in 1:n
            if value(y[i, j]) > 0.5
                score = comp[(i, j)]
                println("Donor $(i) donates to Patient $(j) with score $(score)")
                total_score += score
                push!(matches, (Donor = i, Patient = j, Score = score))
            end
        end
    end
    println("\nTotal Matching Score: ", total_score)
    # display(matches)
else
    println("\nNo optimal solution found. Status: ", status)
end


Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2553220
Academic license 2553220 - for non-commercial use only - registered to ga___@mit.edu
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 9 7940HS w/ Radeon 780M Graphics, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Academic license 2553220 - for non-commercial use only - registered to ga___@mit.edu
Optimize a model with 10200 rows, 10000 columns and 30000 nonzeros
Model fingerprint: 0x98ec0bec
Variable types: 0 continuous, 10000 integer (10000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e+00, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 510.0000000
Presolve removed 10014 rows and 7240 columns
Presolve time: 0.02s
Presolved: 186 rows, 2760 columns, 5520 nonzeros
Vari

# Introducing Transportation Cost

# Introducing Surgery Constrains